In [1]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
spark.conf.set("spark.sql.parquet.vorder.enabled","true")
spark.conf.set("spark.microsoft.delta.optimizeWrite.enabled", "true")
spark.conf.set("spark.microsoft.delta.optimizeWrite.binSize", "1073741824")

from pyspark.sql.types import IntegerType,BooleanType,DateType

StatementMeta(, 907b027b-e344-4d33-b4cc-799803f36384, 3, Finished, Available)

In [3]:
df = spark.read.format("parquet").load('Files/TPCH-CUR/customers/*')
display(df.limit(10))

StatementMeta(, e7e8294d-b94b-4c1c-8b0e-efdd23001bf3, 5, Finished, Available)

SynapseWidget(Synapse.DataFrame, 3c4eb999-8e28-4462-82be-b02628a785fd)

In [4]:
df = spark.read.format("parquet").load('Files/TPCH-CUR/orders/orders.tbl.1.parquet')
display(df.limit(100))

StatementMeta(, e7e8294d-b94b-4c1c-8b0e-efdd23001bf3, 6, Finished, Available)

SynapseWidget(Synapse.DataFrame, 9e18ebcb-02f1-48c2-89c1-ac2a736851fc)

In [3]:
from pyspark.sql.functions import col, year, month, quarter

table_name = 'fact_orders_F4'

df = spark.read.format("parquet").load('Files/TPCH-CUR/orders/*')
df = df.withColumn('Year', year(col("orderdate")))
df = df.withColumn('Quarter', quarter(col("orderdate")))
df = df.withColumn('Month', month(col("orderdate")))

df.write.mode("overwrite").format("delta").partitionBy("Year","Quarter").save("Tables/" + table_name)


StatementMeta(, 0af87402-74c1-42b2-8577-a30d95587a8c, 5, Finished, Available)

In [2]:
df = spark.sql("SELECT count(*) FROM Demo_Lakehouse_2CU.fact_orders_F4")
display(df)

StatementMeta(, c733b724-a9f7-4716-ae2f-a4c8f40f85ed, 4, Finished, Available)

SynapseWidget(Synapse.DataFrame, 2fbdeed1-f0dd-477d-af38-a68cc3aa3d9b)

In [3]:
df = spark.sql('SELECT COUNT(1) from Demo_Lakehouse_2CU.fact_orders')
display(df)

StatementMeta(, 5a42c9d3-ee9e-48c5-83c7-2a4cf1ce1d3b, 5, Finished, Available)

In [1]:
df = spark.sql("SELECT Year, Quarter, Month, COUNT(orderkey) FROM Demo_Lakehouse_2CU.fact_orders group by Year, Quarter, Month order by Year, Quarter, Month")
display(df)

StatementMeta(, 5a42c9d3-ee9e-48c5-83c7-2a4cf1ce1d3b, 3, Finished, Available)

SynapseWidget(Synapse.DataFrame, 3dac1a99-e26e-4dda-870f-6c6c09bc15a0)

In [2]:
table_name = 'dim_customers'

df = spark.read.format("parquet").load('Files/TPCH-CUR/customers/*')
df.withColumn("CUSTKEY",df.CUSTKEY.cast(IntegerType()))

df.write.mode("overwrite").format("delta").save("Tables/" + table_name)

StatementMeta(, 907b027b-e344-4d33-b4cc-799803f36384, 4, Finished, Available)

In [10]:
table_name = 'dim_nation'

df = spark.read.format("parquet").load('Files/TCPH-RAW/Nation/*')

df.write.mode("overwrite").format("delta").save("Tables/" + table_name)

StatementMeta(, 2643dcf8-1a74-4cd0-b7fd-be07929134d4, 12, Finished, Available)

In [11]:
table_name = 'dim_region_c'

df = spark.read.format("parquet").load('Files/TCPH-RAW/Region/*')

df.write.mode("overwrite").option("overwriteSchema","true").format("delta").save("Tables/" + table_name)

StatementMeta(, 2643dcf8-1a74-4cd0-b7fd-be07929134d4, 13, Finished, Available)

In [12]:
from pyspark.sql.functions import col, year, month, quarter

table_name = 'fact_orderline'

dforder = spark.read.format("parquet").load('Files/TCPH-RAW/Orders/*')
dforder = dforder.withColumn('Year', year(col("o_orderdate")))
dforder = dforder.withColumn('Quarter', quarter(col("o_orderdate")))
dforder = dforder.withColumn('Month', month(col("o_orderdate")))

dfline = spark.read.format("parquet").load('Files/TCPH-RAW/LineItems/*')

dforderline = dforder.join(dfline,dforder.o_orderkey == dfline.l_orderkey, how='Inner')

dforderline.write.mode("overwrite").format("delta").partitionBy("Year","Quarter").save("Tables/" + table_name)

StatementMeta(, 2643dcf8-1a74-4cd0-b7fd-be07929134d4, 14, Finished, Available)